In [6]:
from src.data_collection.p05 import get_p05
from src.utils import get_base_url

import re
import requests
import pandas as pd
import zipfile

from bs4 import BeautifulSoup
from io import StringIO
from io import BytesIO
from logging import getLogger
from typing import List, Optional, Union
from urllib.parse import urlparse
from urllib.request import urlopen
from zipfile import ZipFile



In [8]:
cig_2024_url = "https://dati.anticorruzione.it/opendata/dataset/cig"

base_link = get_base_url(cig_2024_url)

html_page = requests.get(cig_2024_url)
soup = BeautifulSoup(html_page.content, 'html.parser')

In [13]:
files_title = "cig_csv"
file_urls = soup.findAll(
    'a', 
    class_="heading",
    title=re.compile(f"{files_title}")
)
links = [link.get('href') for link in file_urls]

In [14]:
links

['/opendata/dataset/cig/resource/7c358621-d192-4859-a988-3038f923b8a9',
 '/opendata/dataset/cig/resource/94cff802-405d-4e1a-a895-bfc5a8c9081c',
 '/opendata/dataset/cig/resource/78e54bab-19d2-4cb2-a159-425ee8ee98e9',
 '/opendata/dataset/cig/resource/c087b461-8bbe-4313-9f7f-4270db12c90a',
 '/opendata/dataset/cig/resource/76279de6-f702-4d16-b0d2-fbcbe6be1d32',
 '/opendata/dataset/cig/resource/aa87d5b4-970a-4cca-b146-99e88e503a0d']

In [15]:
data_links = []

for link in links:
    l = base_link+link
    html_page = requests.get(l)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    file_urls = soup.findAll(
        'a',
        class_="btn btn-primary resource-url-analytics resource-type-None"
    )
    data_links.append(file_urls[0].get('href'))

data_links

['https://dati.anticorruzione.it/opendata/download/dataset/cig/filesystem/20240201-cig_csv.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig/filesystem/20240401-cig_csv.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig/filesystem/20240501-cig_csv.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig/filesystem/20240601-cig_csv.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig/filesystem/20240701-cig_csv.zip',
 'https://dati.anticorruzione.it/opendata/download/dataset/cig/filesystem/cig_csv_logCsv.csv']

In [16]:
resp = urlopen(data_links[0])
zipfile = ZipFile(BytesIO(resp.read()))
fname = zipfile.namelist()[0]
df = pd.read_csv(zipfile.open(fname), dtype=object, delimiter=';')

df.head()

,cig,cig_accordo_quadro,numero_gara,oggetto_gara,importo_complessivo_gara,n_lotti_componenti,oggetto_lotto,importo_lotto,oggetto_principale_contratto,stato,...,TIPO_APPALTO_RISERVATO,CUI_PROGRAMMA,FLAG_PREV_RIPETIZIONI,COD_IPOTESI_COLLEGAMENTO,IPOTESI_COLLEGAMENTO,CIG_COLLEGAMENTO,COD_ESITO,ESITO,DATA_COMUNICAZIONE_ESITO,FLAG_PNRR_PNC
0,A023D14C15,NaN,9399405,OF 1130 - FORNITURA DI CAVI IN FIBRA OTTICA,43322179.64,9,"LOTTO 6 - FORNITURA DI MICROCAVI ADSS, ADSS LIGHT",3916726.62,FORNITURE,ATTIVO,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1
1,A023D2E18D,NaN,9399358,LAVORI DI REVAMPING E MESSA IN SICUREZZA DEGLI...,129434.06,1,LAVORI DI REVAMPING E MESSA IN SICUREZZA DEGLI...,129434.06,LAVORI,ATTIVO,...,NaN,L037923080283202100019,0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,A023F3D471,NaN,9399849,RICHIESTA RINNOVO DETERMINAZIONE N. 417/2018 D...,300000.0,1,RICHIESTA RINNOVO DETERMINAZIONE N. 417/2018 D...,300000.0,FORNITURE,ATTIVO,...,NaN,F10771180014202300150,0,1.0,RIPETIZIONE DI LAVORI O SERVIZI ANALOGHI,70414081B4,NaN,NaN,NaN,0
3,A0240982CC,NaN,9400243,FORNITURA SISTEMI DI TELERILEVAMENTO DA DRONE ...,401639.34,2,"SISTEMI DI TELERILEVAMENTO DA DRONE, LAPTOP E ...",114754.1,FORNITURE,ATTIVO,...,NaN,F00055590327202300024,0,NaN,NaN,NaN,NaN,NaN,NaN,1
4,A0241DEFCE,9930363AC7,9400621,SERVIZIO DI GESTIONE DELLA CONSULTAZIONE E VEN...,650000.0,1,SERVIZIO DI GESTIONE DELLA CONSULTAZIONE E VEN...,650000.0,SERVIZI,ATTIVO,...,NaN,S80062590379202300027,0,NaN,NaN,NaN,1.0,AGGIUDICATA,2023-12-20,0


In [18]:
df['data_pubblicazione']

0        2023-11-03
1        2023-10-23
2        2023-10-27
3        2023-11-13
4        2023-11-08
            ...    
73344    2023-11-08
73345    2023-10-30
73346    2023-10-30
73347    2023-11-07
73348    2023-10-26
Name: data_pubblicazione, Length: 73349, dtype: object

In [19]:
# this should break as it's not a data link 
resp = urlopen(data_links[-1])
zipfile = ZipFile(BytesIO(resp.read()))
fname = zipfile.namelist()[0]
df = pd.read_csv(zipfile.open(fname), dtype=object, delimiter=';')

df.head()

BadZipFile: File is not a zip file

In [2]:
p05 = get_p05()

In [3]:
p05.shape

(210264, 61)

In [4]:
p05.columns

Index(['cig', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato', 'settore',
       'luogo_istat', 'provincia', 'data_pubblicazione',
       'data_scadenza_offerta', 'cod_tipo_scelta_contraente',
       'tipo_scelta_contraente', 'cod_modalita_realizzazione',
       'modalita_realizzazione', 'codice_ausa',
       'cf_amministrazione_appaltante',
       'denominazione_amministrazione_appaltante', 'sezione_regionale',
       'id_centro_costo', 'denominazione_centro_costo', 'anno_pubblicazione',
       'mese_pubblicazione', 'cod_cpv', 'descrizione_cpv', 'flag_prevalente',
       'COD_MOTIVO_CANCELLAZIONE', 'MOTIVO_CANCELLAZIONE',
       'DATA_CANCELLAZIONE', 'DATA_ULTIMO_PERFEZIONAMENTO',
       'COD_MODALITA_INDIZIONE_SPECIALI', 'MODALITA_INDIZIONE_SPECIALI',
       'COD_MODALITA_INDIZIONE_SERVIZI', 'MODALITA_INDIZIONE_SERVIZI',
       'DURATA_PREVIST

In [5]:
p05.head()

,cig,cig_accordo_quadro,numero_gara,oggetto_gara,importo_complessivo_gara,n_lotti_componenti,oggetto_lotto,importo_lotto,oggetto_principale_contratto,stato,...,TIPO_APPALTO_RISERVATO,CUI_PROGRAMMA,FLAG_PREV_RIPETIZIONI,COD_IPOTESI_COLLEGAMENTO,IPOTESI_COLLEGAMENTO,CIG_COLLEGAMENTO,COD_ESITO,ESITO,DATA_COMUNICAZIONE_ESITO,FLAG_PNRR_PNC
0,9766166F1F,NaN,9042660,'AVVISO INVESTIMENTO 1.2 'ABILITAZIONE AL CLOU...,9800.0,1,'AVVISO INVESTIMENTO 1.2 'ABILITAZIONE AL CLOU...,9800.0,SERVIZI,ATTIVO,...,NaN,00104440144202300001,1,NaN,NaN,NaN,NaN,NaN,NaN,1
1,A019FC8DE0,7194294737,9362129,PNRR MISSIONE 6 COMPONENTE 2 INVESTIMENTO 1.1....,225000.0,1,PNRR MISSIONE 6 COMPONENTE 2 INVESTIMENTO 1.1....,225000.0,FORNITURE,ATTIVO,...,NaN,0000000000000000000000,0,NaN,NaN,NaN,NaN,NaN,NaN,1
2,819779499C,NaN,7678766,S.C. 20178924 QUADRI DI BASSA TENSIONE PER CAB...,8280000.0,2,S.C. 20178924 SCOMPARTO DI BASSA TENSIONE TIPO...,3312000.0,FORNITURE,ATTIVO,...,NaN,NaN,1,NaN,NaN,NaN,1.0,AGGIUDICATA,2023-08-29,1
3,8226615176,7040391A70,7700533,FORNITURA CUCINE DI REPARTO,50000.0,1,FORNITURA CUCINE DI REPARTO,50000.0,FORNITURE,ATTIVO,...,NaN,NaN,0,NaN,NaN,NaN,1.0,AGGIUDICATA,2021-01-22,1
4,82066050B0,NaN,7686388,GARA GBO PER INAIL - ID 2208,9693890.0,2,LOTTO 2 - GARA GBO PER INAIL ID 2208,1703390.0,SERVIZI,ATTIVO,...,NaN,NaN,0,NaN,NaN,NaN,1.0,AGGIUDICATA,2022-01-18,1
